In [1]:
import tensorflow as tf
from tensorflow import keras
import keras_tuner as kt
import numpy as np

2024-01-10 09:42:22.971548: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-10 09:42:23.049884: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-10 09:42:23.050023: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-10 09:42:23.051127: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-10 09:42:23.059622: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-10 09:42:23.060111: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
from PIL import Image
import os

def get_and_save_data_and_labels(path):
    data = open(path,"r")

    names = []
    types = []

    for line in data:
        fields = line.split(" ")
        names.append( fields[0] )
        types.append( fields[1] )
        
    nData = len(names)
    imgSize = 128
    galaxies = np.zeros((nData,imgSize,imgSize,3))
    labels = np.zeros(nData, dtype='int')

    for i in range(nData):
        filename = "/mnt/share/nnds/galaxies/png/"+str(names[i])+".png"
        img = Image.open(filename)

        imgResized = img.resize(size=(imgSize,imgSize))
        imgArr = np.array(imgResized)

        imgArr = imgArr/255.
        
        galaxies[i,:,:,:] = imgArr 
        labels[i] = types[i]
    
    split = 955

    train_data = galaxies[:split,:,:,:]
    train_labels = labels[:split]

    test_data = galaxies[split+1:nData-1,:,:,:]
    test_labels = labels[split+1:nData-1]
    
    os.makedirs("galaxies")
    np.save("galaxies/trainGalaxies.npy", train_data)
    np.save("galaxies/trainLabels.npy", train_labels)

    np.save("galaxies/testGalaxies.npy", test_data)
    np.save("galaxies/testLabels.npy", test_labels)
        
    return train_data, train_labels, test_data, test_labels

In [3]:
def load_train_and_test_data():
    train_data = np.load("galaxies/trainGalaxies.npy")
    train_labels = np.load("galaxies/trainLabels.npy", )

    test_data = np.load("galaxies/testGalaxies.npy")
    test_labels = np.load("galaxies/testLabels.npy", )
    
    return train_data, train_labels, test_data, test_labels

In [4]:
# 1a.
train_data, train_labels, test_data, test_labels = load_train_and_test_data()

In [5]:
galNet = keras.Sequential([
    keras.layers.Conv2D(96, (6, 6), activation='relu', input_shape=(128,128,3)),
    keras.layers.MaxPooling2D(pool_size=(3,3)),
    keras.layers.Flatten(),
    keras.layers.Dense(24, activation='relu'),
    keras.layers.Dense(3, activation='softmax')
])

2024-01-10 09:42:41.381497: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-01-10 09:42:41.383222: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [6]:
galNet.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 123, 123, 96)      10464     
                                                                 
 max_pooling2d (MaxPooling2  (None, 41, 41, 96)        0         
 D)                                                              
                                                                 
 flatten (Flatten)           (None, 161376)            0         
                                                                 
 dense (Dense)               (None, 24)                3873048   
                                                                 
 dense_1 (Dense)             (None, 3)                 75        
                                                                 
Total params: 3883587 (14.81 MB)
Trainable params: 3883587 (14.81 MB)
Non-trainable params: 0 (0.00 Byte)
________________

In [7]:
# 1b.
epochs = 50
galNet.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])
history = galNet.fit(train_data, train_labels, validation_data=(test_data, test_labels), epochs=epochs)

Epoch 1/50


2024-01-10 09:43:11.016538: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 187760640 exceeds 10% of free system memory.
2024-01-10 09:43:12.867835: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 185905152 exceeds 10% of free system memory.
2024-01-10 09:43:12.971308: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 185905152 exceeds 10% of free system memory.


 1/30 [>.............................] - ETA: 55s - loss: 1.0924 - accuracy: 0.4688

2024-01-10 09:43:13.400762: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 185905152 exceeds 10% of free system memory.
2024-01-10 09:43:13.458129: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 185905152 exceeds 10% of free system memory.


30/30 [==============================] - 16s 470ms/step - loss: 1.1438 - accuracy: 0.4806 - val_loss: 0.9945 - val_accuracy: 0.5042
Epoch 2/50
30/30 [==============================] - 12s 409ms/step - loss: 0.8928 - accuracy: 0.5717 - val_loss: 0.8646 - val_accuracy: 0.6471
Epoch 3/50
30/30 [==============================] - 8s 263ms/step - loss: 0.7229 - accuracy: 0.6785 - val_loss: 0.7136 - val_accuracy: 0.6639
Epoch 4/50
30/30 [==============================] - 9s 284ms/step - loss: 0.6280 - accuracy: 0.6764 - val_loss: 0.6734 - val_accuracy: 0.6471
Epoch 5/50
30/30 [==============================] - 12s 394ms/step - loss: 0.5482 - accuracy: 0.6932 - val_loss: 0.5611 - val_accuracy: 0.6723
Epoch 6/50
30/30 [==============================] - 10s 326ms/step - loss: 0.4696 - accuracy: 0.7738 - val_loss: 0.5120 - val_accuracy: 0.7899
Epoch 7/50
30/30 [==============================] - 12s 404ms/step - loss: 0.4247 - accuracy: 0.8366 - val_loss: 0.4435 - val_accuracy: 0.8403
Epoch 8/50
3

In [16]:
# 1c.
predictions = galNet.predict(test_data)
predicted_labels = np.argmax(predictions, axis=1)
correct_counts = [0, 0, 0]
total_counts = [0, 0, 0]

for i in range(len(test_labels)):
    true_label = test_labels[i]
    predicted_label = predicted_labels[i]

    total_counts[true_label] += 1

    if true_label == predicted_label:
        correct_counts[true_label] += 1
        
for i in range(3):
    accuracy = correct_counts[i] / total_counts[i] if total_counts[i] > 0 else 0
    print(f"Class {i} accuracy: {accuracy}")

print(f"Training data distribution: {np.bincount(train_labels)}")
print(f"Test data distribution: {np.bincount(test_labels)}")

4/4 [==============================] - 0s 35ms/step
Class 0 accuracy: 0.9333333333333333
Class 1 accuracy: 0.7666666666666667
Class 2 accuracy: 0.7241379310344828
Training data distribution: [238 515 202]
Test data distribution: [30 60 29]


In [1]:
# 2a.
from keras.datasets import cifar10
from keras.utils import to_categorical
import tensorflow as tf
from tensorflow import keras

2024-02-07 15:02:51.454587: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-07 15:02:51.482959: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-07 15:02:51.482981: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-07 15:02:51.483560: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-07 15:02:51.487579: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-07 15:02:51.487912: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [5]:
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

train_images, test_images = train_images / 255.0, test_images / 255.0

train_images = train_images.reshape((-1, 32, 32, 3))
test_images = test_images.reshape((-1, 32, 32, 3))

train_labels = to_categorical(train_labels, num_classes=10)
test_labels = to_categorical(test_labels, num_classes=10)

In [6]:
# 2b. 
model = keras.Sequential([
    keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(64, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])

In [7]:
# 2c. training for 20 epochs
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_images, train_labels, epochs=20, validation_split=0.2)

Epoch 1/20
1250/1250 [==============================] - 8s 6ms/step - loss: 1.5221 - accuracy: 0.4464 - val_loss: 1.2709 - val_accuracy: 0.5522
Epoch 2/20
1250/1250 [==============================] - 7s 5ms/step - loss: 1.1679 - accuracy: 0.5886 - val_loss: 1.1326 - val_accuracy: 0.6027
Epoch 3/20
1250/1250 [==============================] - 7s 5ms/step - loss: 1.0282 - accuracy: 0.6408 - val_loss: 1.0176 - val_accuracy: 0.6471
Epoch 4/20
1250/1250 [==============================] - 7s 5ms/step - loss: 0.9344 - accuracy: 0.6755 - val_loss: 0.9698 - val_accuracy: 0.6658
Epoch 5/20
1250/1250 [==============================] - 7s 5ms/step - loss: 0.8719 - accuracy: 0.6952 - val_loss: 0.9412 - val_accuracy: 0.6787
Epoch 6/20
1250/1250 [==============================] - 7s 5ms/step - loss: 0.8094 - accuracy: 0.7190 - val_loss: 0.9423 - val_accuracy: 0.6786
Epoch 7/20
1250/1250 [==============================] - 7s 5ms/step - loss: 0.7619 - accuracy: 0.7347 - val_loss: 0.9284 - val_accuracy:

### 2d.
Image augmentation can be useful to improve model generalization. Some common augmentations for this dataset include:
- Random horizontal flip: Reflecting an image horizontally.
- Random rotation: Rotating the image by a certain degree.
- Width and height shift: Shifting the image horizontally or vertically.
- Zoom range: Zooming into or out of the image.

```py
# Example:
datagen = keras.preprocessing.image.ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range=0.1
)

# Applying augmentation during training
augmented_train_data = datagen.flow(train_images, train_labels)

model.fit(augmented_train_data, epochs=20, validation_data=(test_images, test_labels))
```